In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pre_process import pre_processing

In [4]:
sequence_length = 20
X, a, y = pre_processing('1_PnL.csv','with_Indicator_1h_BTC_USDT.csv', sequence_length)

c:\Users\Shubham Garg\Documents\Internship@SAI\Day1\pre_process.py:43: RuntimeWarning: invalid value encountered in cast
  return buy_sequences.astype(np.int32), attention_masks.astype(np.int32), targets.astype(np.int32)


In [9]:
X_train = X[2:1100]
y_train = y[2:1100]
a_train = a[2:1100]
X_test = X[1100:1200]
y_test = y[1100:1200]
a_test = a[1100:1200]

In [10]:
X_train[2]

array([  1,   2,   1,   1,   1,   1,  65,   1,   1,   1,   5,   1,   3,
         2,   1,   1,  66,   2,   1,   2,   5,   9,   8,   4,   1,   1,
        66,   3,   2,   3,   8,   5,   8,   7,   1,   1,  66,   5,   4,
         5,  11,  98,  14,  99,  28,   7,  26,  26,  16,  11,  99, 100,
        91,  91,  42,  13,  12,  40,  26,  18,  91,  93,  99,  97,  54,
        19,   4,  53,  36,  24,  96,  94,  88,  85,  62,  25,   1,  61,
        43,  30,  87,  81,  82,  80,  68,  31,   2,  67,  50,  36,  81,
        82,  88,  82,  74,  37,   4,  71,  55,  41,  82,  91,  90,  90,
        80,  43,   7,  77,  62,  47,  90,  95,  98, 100,  86,  50,  11,
        83,  68,  53, 100,  92,  91,  88,  90,  56,  17,  86,  73,  59,
        88,  81,  90,  90,  93,  62,  26,  88,  78,  65,  90,  86,  96,
        92,  95,  69,  36,  91,  82,  71,  92,  85,  98,  90,  97,  75,
        47,  92,  86,  76,  90,  85,  95,  93,  98,  82,  60,  94,  89,
        82,  93,  88,  96,  91,  99,  88,  72,  95,  93,  88,  9

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': X_train, 'attention_mask': a_train},
    y_train
)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': X_test, 'attention_mask': a_test},
    y_test
)).batch(32)

In [12]:
from transformers import TFDistilBertForSequenceClassification, AdamWeightDecay
model_name = "distilbert-base-uncased"
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [13]:
# Define optimizer
optimizer = AdamWeightDecay(learning_rate=1e-5, weight_decay_rate=0.01)

# Compile the model, explicitly setting 'from_logits=True'
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # Explicitly use SparseCategoricalCrossentropy and set from_logits=True
              metrics=["accuracy"])

# Train the model
model.fit(
    train_dataset,
    epochs=10,
    batch_size=32,
    validation_data=test_dataset,
    steps_per_epoch=len(train_dataset),
    validation_steps=len(test_dataset)
)

Epoch 1/10
35/35 [==============================] - 33s 698ms/step - loss: 0.8866 - accuracy: 0.6630 - val_loss: 0.8159 - val_accuracy: 0.7000
Epoch 2/10
35/35 [==============================] - 23s 648ms/step - loss: 0.8447 - accuracy: 0.6785 - val_loss: 0.8046 - val_accuracy: 0.7000
Epoch 3/10
35/35 [==============================] - 23s 659ms/step - loss: 0.8385 - accuracy: 0.6785 - val_loss: 0.8057 - val_accuracy: 0.7000
Epoch 4/10
35/35 [==============================] - 23s 652ms/step - loss: 0.8399 - accuracy: 0.6785 - val_loss: 0.8050 - val_accuracy: 0.7000
Epoch 5/10
35/35 [==============================] - 22s 636ms/step - loss: 0.8387 - accuracy: 0.6785 - val_loss: 0.8034 - val_accuracy: 0.7000
Epoch 6/10
35/35 [==============================] - 23s 656ms/step - loss: 0.8365 - accuracy: 0.6785 - val_loss: 0.8043 - val_accuracy: 0.7000
Epoch 7/10
35/35 [==============================] - 22s 637ms/step - loss: 0.8367 - accuracy: 0.6785 - val_loss: 0.8039 - val_accuracy: 0.7000